In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import sklearn
from IPython.display import display
%matplotlib inline

In [2]:
# Example of using the `score` method to evaluate supervised model

from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
# create a synthetic dataset
X, y = make_blobs(random_state=0)
# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 
# instantiate a model and fit it to the training set
logreg = LogisticRegression().fit(X_train, y_train)
# evaluate the model on the test set
print("Test set score: {:.2f}".format(logreg.score(X_test, y_test)))

Test set score: 0.88


In [3]:
# Try cross-validation in scikit-learn
# Three-fold cross-validation is the default.
from sklearn.model_selection import cross_val_score 
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
iris = load_iris()
logreg = LogisticRegression()
scores = cross_val_score(logreg, iris.data, iris.target) 
print("Cross-validation scores: {}".format(scores))

Cross-validation scores: [ 0.96078431  0.92156863  0.95833333]


In [4]:
# Try changing to 5-fold
scores = cross_val_score(logreg, iris.data, iris.target, cv=5) 
print("Cross-validation scores: {}".format(scores))

Cross-validation scores: [ 1.          0.96666667  0.93333333  0.9         1.        ]


In [5]:
# Summarize accuracies by computing the mean.
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.96


In [6]:
# Try implementing a grid search
# naive grid search implementation
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print("Size of training set: {} size of test set: {}".format(
          X_train.shape[0], X_test.shape[0]))
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]: 
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the test set
        score = svm.score(X_test, y_test)
        # if we got a better score, store the score and parameters 
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
print("Best score: {:.2f}".format(best_score)) 
print("Best parameters: {}".format(best_parameters))

Size of training set: 112 size of test set: 38
Best score: 0.97
Best parameters: {'C': 100, 'gamma': 0.001}


In [9]:
# More sophisticatedly, split the data in training, validation, and test sets.
from sklearn.svm import SVC
# split data into train+validation set and test set 
X_trainval, X_test, y_trainval, y_test = train_test_split(
iris.data, iris.target, random_state=0)
# split train+validation set into training and validation sets 
X_train, X_valid, y_train, y_valid = train_test_split(
X_trainval, y_trainval, random_state=1)
print("Size of training set: {} size of validation set: {} size of test set:"
" {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0])) 
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]: 
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the validation set
        score = svm.score(X_valid, y_valid)
        # if we got a better score, store the score and parameters 
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
            
# rebuild a model on the combined training and validation set, 
# and evaluate it on the test set
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score)) 
print("Best parameters: ", best_parameters)
print("Test set score with best parameters: {:.2f}".format(test_score))

Size of training set: 84 size of validation set: 28 size of test set: 38

Best score on validation set: 0.96
Best parameters:  {'C': 10, 'gamma': 0.001}
Test set score with best parameters: 0.92


In [10]:
# Still more sophisticatedly, split the dataset 3 ways and use cross-validation
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]: 
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
    # for each combination of parameters, # train an SVC
        svm = SVC(gamma=gamma, C=C)
        # perform cross-validation
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5) 
        # compute mean cross-validation accuracy
        score = np.mean(scores)
        # if we got a better score, store the score and parameters 
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
# rebuild a model on the combined training and validation set 
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
# In scikit-learn there are built-in methods to perform grid search with cross-validation
# Need specify parameter grid using a dictionary

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                  'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
print("Parameter grid:\n{}".format(param_grid))

from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

X_train, X_test, y_train, y_test = train_test_split(
        iris.data, iris.target, random_state=0)
# The grid_search object behaves like a classifier and can call predict, fit, and score on it
# But when call fit, it will run cross-validation for each combination of parameters
grid_search.fit(X_train, y_train)

Parameter grid:
{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [14]:
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.97
Best parameters: {'C': 100, 'gamma': 0.01}
Best cross-validation score: 0.97
Best estimator:
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [15]:
# Try visualizing grid search using pandas dataframe
import pandas as pd
# convert to DataFrame
results = pd.DataFrame(grid_search.cv_results_) # show the first 5 rows
display(results.head())

mean_fit_time  mean_score_time  mean_test_score  mean_train_score param_C  \
0       0.000510         0.000222         0.366071          0.366079   0.001   
1       0.000533         0.000205         0.366071          0.366079   0.001   
2       0.000489         0.000193         0.366071          0.366079   0.001   
3       0.000459         0.000178         0.366071          0.366079   0.001   
4       0.000514         0.000195         0.366071          0.366079   0.001   

  param_gamma                        params  rank_test_score  \
0       0.001  {'C': 0.001, 'gamma': 0.001}               22   
1        0.01   {'C': 0.001, 'gamma': 0.01}               22   
2         0.1    {'C': 0.001, 'gamma': 0.1}               22   
3           1      {'C': 0.001, 'gamma': 1}               22   
4          10     {'C': 0.001, 'gamma': 10}               22   

   split0_test_score  split0_train_score       ...         split2_test_score  \
0              0.375            0.363636       ...                  0.363636   
1              0.375            0.363636       ...                  0.363636   
2              0.375            0.363636       ...                  0.363636   
3              0.375            0.363636       ...                  0.363636   
4              0.375            0.363636       ...                  0.363636   

   split2_train_score  split3_test_score  split3_train_score  \
0            0.366667           0.363636            0.366667   
1            0.366667           0.363636            0.366667   
2            0.366667           0.363636            0.366667   
3            0.366667           0.363636            0.366667   
4            0.366667           0.363636            0.366667   

   split4_test_score  split4_train_score  std_fit_time  std_score_time  \
0           0.380952            0.362637      0.000096        0.000067   
1           0.380952            0.362637      0.000070        0.000020   
2           0.380952            0.362637      0.000042        0.000022   
3           0.380952            0.362637      0.000014        0.000003   
4           0.380952            0.362637      0.000032        0.000008   

   std_test_score  std_train_score  
0        0.011371         0.002852  
1        0.011371         0.002852  
2        0.011371         0.002852  
3        0.011371         0.002852  
4        0.011371         0.002852  

[5 rows x 22 columns]

In [16]:
# Visualize as a heat map.
scores = np.array(results.mean_test_score).reshape(6, 6)
# plot the mean cross-validation scores
# color coded by accuracy
mglearn.tools.heatmap(scores, xlabel='gamma', xticklabels=param_grid['gamma'],
            ylabel='C', yticklabels=param_grid['C'], cmap="viridis")

In [17]:
fig, axes = plt.subplots(1, 3, figsize=(13, 5))
param_grid_linear = {'C': np.linspace(1, 2, 6),
                         'gamma':  np.linspace(1, 2, 6)}
param_grid_one_log = {'C': np.linspace(1, 2, 6),
                          'gamma':  np.logspace(-3, 2, 6)}
param_grid_range = {'C': np.logspace(-3, 2, 6),
                        'gamma':  np.logspace(-7, -2, 6)}
for param_grid, ax in zip([param_grid_linear, param_grid_one_log, param_grid_range], axes):
    grid_search = GridSearchCV(SVC(), param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    scores = grid_search.cv_results_['mean_test_score'].reshape(6, 6)
    # plot the mean cross-validation scores
    scores_image = mglearn.tools.heatmap(
            scores, xlabel='gamma', ylabel='C', xticklabels=param_grid['gamma'],
            yticklabels=param_grid['C'], cmap="viridis", ax=ax)
plt.colorbar(scores_image, ax=axes.tolist())

In [18]:
# Can do conditional grid 

param_grid = [{'kernel': ['rbf'],
                   'C': [0.001, 0.01, 0.1, 1, 10, 100],
                   'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
                  {'kernel': ['linear'],
            'C': [0.001, 0.01, 0.1, 1, 10, 100]}] 
print("List of grids:\n{}".format(param_grid))

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

List of grids:
[{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['rbf']}, {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear']}]
Best parameters: {'C': 100, 'kernel': 'rbf', 'gamma': 0.01}
Best cross-validation score: 0.97


In [19]:
results = pd.DataFrame(grid_search.cv_results_)
# we display the transposed table so that it better fits on the page: 
display(results.T)

0   \
mean_fit_time                                         0.000478697   
mean_score_time                                       0.000183392   
mean_test_score                                          0.366071   
mean_train_score                                         0.366079   
param_C                                                     0.001   
param_gamma                                                 0.001   
param_kernel                                                  rbf   
params              {'C': 0.001, 'kernel': 'rbf', 'gamma': 0.001}   
rank_test_score                                                27   
split0_test_score                                           0.375   
split0_train_score                                       0.363636   
split1_test_score                                        0.347826   
split1_train_score                                       0.370787   
split2_test_score                                        0.363636   
split2_train_score                                       0.366667   
split3_test_score                                        0.363636   
split3_train_score                                       0.366667   
split4_test_score                                        0.380952   
split4_train_score                                       0.362637   
std_fit_time                                          6.34128e-05   
std_score_time                                        1.46385e-05   
std_test_score                                          0.0113708   
std_train_score                                        0.00285176   

                                                              1   \
mean_fit_time                                        0.000467157   
mean_score_time                                      0.000177431   
mean_test_score                                         0.366071   
mean_train_score                                        0.366079   
param_C                                                    0.001   
param_gamma                                                 0.01   
param_kernel                                                 rbf   
params              {'C': 0.001, 'kernel': 'rbf', 'gamma': 0.01}   
rank_test_score                                               27   
split0_test_score                                          0.375   
split0_train_score                                      0.363636   
split1_test_score                                       0.347826   
split1_train_score                                      0.370787   
split2_test_score                                       0.363636   
split2_train_score                                      0.366667   
split3_test_score                                       0.363636   
split3_train_score                                      0.366667   
split4_test_score                                       0.380952   
split4_train_score                                      0.362637   
std_fit_time                                         3.78483e-05   
std_score_time                                       6.01722e-06   
std_test_score                                         0.0113708   
std_train_score                                       0.00285176   

                                                             2   \
mean_fit_time                                       0.000456238   
mean_score_time                                     0.000177193   
mean_test_score                                        0.366071   
mean_train_score                                       0.366079   
param_C                                                   0.001   
param_gamma                                                 0.1   
param_kernel                                                rbf   
params              {'C': 0.001, 'kernel': 'rbf', 'gamma': 0.1}   
rank_test_score                                              27   
split0_test_score                                         0.375   
split0_train_score                  